In [0]:
create or alter proc CreatePyModelRealtimeScoringOnly as
	
	declare @model varbinary(max), @train_script nvarchar(max);
	delete top(1) from models where model_name = 'RevoMMLRealtimeScoring' and language = 'Py';
	
	--The Python script we want to execute
	set @train_script = N'
from microsoftml import rx_logistic_regression, featurize_text, n_gram
from revoscalepy import rx_serialize_model, RxOdbcData, rx_write_object, RxInSqlServer, rx_set_compute_context, RxLocalSeq

connection_string = "Driver=SQL Server;Server=localhost;Database=tpcxbb_1gb;Trusted_Connection=true;"
dest = RxOdbcData(connection_string, table = "models")
 
training_data["tag"] = training_data["tag"].astype("category")

modelpy = rx_logistic_regression(formula = "tag ~ features",
								 data = training_data, 
								 method = "multiClass", 
								 ml_transforms=[featurize_text(language="English",
															   cols=dict(features="pr_review_content"),
															   word_feature_extractor=n_gram(2, weighting="TfIdf"))],
								 train_threads=1)

modelbin = rx_serialize_model(modelpy, realtime_scoring_only = True)
rx_write_object(dest, key_name="model_name", key="RevoMMLRealtimeScoring", value_name="model", value=modelbin, serialize=False, compress=None, overwrite=False)'; --overwrite=false on 2019, true on 2017.

	exec sp_execute_external_script @language = N'Python'
		,@script = @train_script
		,@input_data_1 = N'select * from product_reviews_training_data'
		,@input_data_1_name = N'training_data'
go

# 🐾 Steps
https://docs.microsoft.com/azure/cognitive-services/text-analytics/quickstarts/python, 
https://azure.microsoft.com/services/cognitive-services/text-analytics

In [4]:
import requests
from pprint import pprint
from IPython.display import HTML

subscription_key = "mykey" #https://portal.azure.com/#@fleitasarts.com/resource/subscriptions/eaab21d5-8ecd-4ef0-a0c4-92fac2e22875/resourceGroups/rg4/providers/Microsoft.CognitiveServices/accounts/trollhunters/keys
text_analytics_base_url = "https://eastus2.api.cognitive.microsoft.com/text/analytics/v2.0/"
sentiment_url = text_analytics_base_url + "sentiment"

documents = {"documents" : [
  {"id": "1", "language": "en", "text": "I had a wonderful experience! The rooms were wonderful and the staff was helpful."},
  {"id": "2", "language": "en", "text": "I had a terrible time at the hotel. The staff was rude and the food was awful."},  
  {"id": "3", "language": "es", "text": "Los caminos que llevan hasta Monte Rainier son espectaculares y hermosos."},  
  {"id": "4", "language": "es", "text": "La carretera estaba atascada. Había mucho tráfico el día de ayer."}
]}

headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(sentiment_url, headers=headers, json=documents)
sentiments = response.json()
pprint(sentiments)

{'documents': [{'id': '1', 'score': 0.9708490371704102},
               {'id': '2', 'score': 0.0019068121910095215},
               {'id': '3', 'score': 0.7456425428390503},
               {'id': '4', 'score': 0.334433376789093}],
 'errors': []}


## 🍖 ref Yelp.py next